In [2]:
from Pointcloud.Modules.Alignor import Alignor
from Pointcloud.Modules import Config as config
from Pointcloud.Modules.FileDataset import FileDataset
from Pointcloud.Modules.GraphBuilder import GraphBuilder
from Pointcloud.Modules.Manager import Manager
from Pointcloud.Modules.Model import Patch2NormalModel
from Pointcloud.Modules.Noise import Noise
from Pointcloud.Modules.Object import FilePointcloud, FileMesh, Pointcloud
from Pointcloud.Modules.Preprocessor import Preprocessor
from Pointcloud.Modules.FileDataset import FileDataset, NoiseLevels
from Pointcloud.Modules.Selector import Selector
from Pointcloud.Modules.Utils import GeneralUtils, TorchUtils, SlicedTorchData
from Pointcloud.Modules.Visualize import visualize_coloring, plot_2d

from copy import deepcopy
from deltaconv.geometry import estimate_basis
from igl import (
    avg_edge_length as igl_avg_edge_length,
    barycenter as igl_barycenter,
    bounding_box_diagonal as igl_bounding_box_diagonal,
    per_vertex_normals as igl_per_vertex_normals,
    read_obj as igl_read_obj,
    triangle_triangle_adjacency as igl_triangle_triangle_adjacency,
    write_obj as igl_write_obj
)
from itertools import zip_longest
from meshplot import plot as mp_plot
from gravomg.util import neighbors_from_stiffness
import numpy as np
from numpy import random as np_random
from pathlib import Path
import pytorch_lightning as pl
import matplotlib.pyplot as plt
from robust_laplacian import point_cloud_laplacian
from scipy.spatial import Delaunay,\
    KDTree
from sklearn.preprocessing import normalize as sklearn_preprocessing_normalize
from timeit import timeit
import torch_geometric as tg
from torch_cluster import knn_graph
import torch
from tqdm import tqdm

EXAMPLE_OBJ_FILE = "common-3d-test-models-master/stanford-bunny.obj"
# EXAMPLE_OBJ_FILE = "common-3d-test-models-master/fandisk.obj"
# EXAMPLE_OBJ_FILE = "common-3d-test-models-master/beetle.obj"
# EXAMPLE_OBJ_FILE = "untitled.obj"

In [ ]:
manager = Manager()
pointcloud = Pointcloud.loadObj(EXAMPLE_OBJ_FILE)
prediction = manager.predict(pointcloud, "tb_logs/testmodel/version_22/checkpoints/0-epoch50-00.62.ckpt")

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


KeyboardInterrupt: 

In [ ]:
start = pointcloud.v.numpy()
end = start + prediction.numpy() / 200
plot = mp_plot(start)
plot.add_lines(start, end)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

1

In [ ]:
graphs, RInv = Preprocessor(pointcloud).getPatches()
ds = tg.data.Batch.from_data_list(graphs)
dl = tg.data.DataLoader(
    dataset=ds,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    num_workers=config.NUM_WORKERS,
    persistent_workers=True,
)
batchedOutput = manager.trainer.predict(manager.model, dl)
output = torch.cat(batchedOutput, dim=0)
R = RInv.transpose(dim0=1, dim1=2)
predictions = Alignor.applyRInv(R, output)

c:\Users\ruben\miniconda3\envs\PCDenoiser\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

PREDICT Profiler Report
Profile stats for: records
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         8.82%      40.748ms        99.77%     461.118ms      23.056ms      35.980ms         7.78%     461.386ms      23.069ms            20  
[pl][profile][Strategy]SingleDeviceStrategy.predict_...         8.01%      37.034ms        66.35%     306.651ms      15.333ms      27.699ms         5.99%   

In [ ]:
y = Alignor.applyRInv(RInv, output)
# y = output
gb = GraphBuilder(pointcloud)
gb.generateNormals()
gt = gb.graph.n

_scale = 0.004

start = pointcloud.v
end = start + y * _scale
plot = mp_plot(start.numpy(), shading = {"point_size": _scale})
plot.add_lines(start.numpy(), end.numpy())
plot.add_lines(start.numpy(), (start + gt * _scale).numpy(), shading={"line_color": "green"})

print((gt * y).sum(dim=1))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

tensor([-0.9678, -0.8555, -0.8999,  ..., -0.8944, -0.2505,  0.3300])
